In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, date

## CFM 101: Group Assignment - Python Roboadvisor
### Team Number: 15
### Team Member Names: Landon Trinh, Ethan Zemelman, Jessie Deng
### Team Strategy Chosen: SAFE

## Goal
- Our team has decided to target dynamically building the safest portfolio
- Given a file of unknown stock tickers, our roboadvisor will run from November 25, 2023 to December 4, 2023
- The deviation will be calculated by taking calculating the difference between the final value of the portfolio and the initial portfolio value ($750,000)
- Ultimately, our aim is to have our portfolio deviate in nominal value. In other words, in the end, we want our portfolio value to have changed as little as possible


## Introduction
> "Theory will only take you so far." - J. Robert Oppenheimer

In theory, our trading strategy should produce a portfolio that deviates in nominal value the least by taking into consideration the following:
- Beta
- Diversification
- Correlation
- Standard deviation
- Expected returns

Our goal will be to tell a convincing story to WHY we are picking our stocks. We will calculate and discuss statistics, display and intepret graphs, and explain our thought process

## 1. Setup
Before implementing our trading strategy, we will initialize required and useful constants as part of the rules:
- Currency of valid stocks (USD or CAD)
- Required average monthly volume (150,000 shares)
- The number of stocks we wish to purchase on the start date (10-22 stocks)
- Time interval (Janurary 1, 2023 - October 31, 2023)
- Minimum number of trading days for month (18 days)
- Minimum stock weighting: $\frac{100}{2n}$%, $n$ = number of stocks in portfolio
- Maximum stock weighting: 20%
- Initial investment amount: $750,000 CAD
- Buying date of roboadvisor: November 25, 2023 - December 4, 2023
- Trading fee for each stock trade: $4.95 CAD

In the end, our roboadvisor should create two DataFrames:

1. $\verb|Portfolio_Final|\\$
- Index: Starts at 1 and ends at number of stocks in portfolio
- Headings: Ticker, Price (price of stock on Nov 25), Currency (CAD or USD), Shares, Value, Weight (adds to 100%)

2. $\verb|Stocks_Final|\\$
We should output this DataFrame to a CSV file titled "Stocks_Group_15.csv"
- Index: Same as "Portfolio Final"
- Headings: Tickers and Shares from "Portfolio_Final"


In [2]:
# Investment amount (CAD)
capital = 750_000

# Number of stocks to buy for portfolio
num_stocks = 15

# Maximum and minimum weightings of each stock in portfolio
min_weight = 1 / (2 * num_stocks)
max_weight = 0.20

# Start and end date for roboadvisor
# start_date = "2023-11-25"
# end_date = "2023-12-04"

# Filtering requirements
valid_currency = ["CAD", "USD"]
min_trading_days = 18
required_avg_volume = 150000

## 2. Filtering
After reading in the CSV file containg stock tickers, we must filter the list of stocks to make sure they are valid stock tickers according to the following rules:
- Include stocks that have an average monthly volume of at leaest 150,000 shares based on Jan 1, 2023 - Oct 31, 2023 (drop any months that don't have at least 18 trading days)
- Stock denominated in USD or CAD

In [3]:
# Read in CSV ticker file
tickers = pd.read_csv("tickers_example.csv", header=None)
tickers = tickers.rename(columns={0: "ticker"})
tickers_lst = tickers["ticker"].tolist()
tickers.head()

,ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


In [4]:
# Set parameters
filter_start_date = "2023-01-01" # Jan 1, 2023
filter_end_date = "2023-10-31" # Oct 31, 2023
filter_interval = "1mo"

In [5]:
# Keep stocks with average monthly volume of 150k shares - drop months with less than 18 trading days
def valid_volume(stock_ticker):

    monthly_volumes = []

    for month in range(1,10):
        # Set monthly date intervals
        month_start_date = str(date(2023, month, 1))
        month_end_date = str(date(2023, month + 1, 1))

        # Retrieve volume data for month
        monthly_volume_hist = stock_ticker.history(interval="1d", start=month_start_date, end=month_end_date).Volume
        monthly_volume_hist.index = monthly_volume_hist.index.strftime("%Y-%m-%d")

        # Retrieve number of trading days and average monthly volume
        trading_days = len(monthly_volume_hist)
        monthly_volume = monthly_volume_hist.sum()

        # Add monthly volume to list if valid number of trading days
        if trading_days >= min_trading_days:
            monthly_volumes.append(monthly_volume)

    # Determine whether average monthly volume meets requirement
    avg_monthly_volume = sum(monthly_volumes) / len(monthly_volumes)
    if avg_monthly_volume < required_avg_volume:
        return False
    return True

In [6]:
# Retrieve filtered tickers
def filter_tickers(tickers):
    filtered_tickers = []
    
    for ticker in tickers:
        try:
            # Get base currency
            stock_ticker = yf.Ticker(ticker)
            base_currency = stock_ticker.fast_info["currency"]
            
            # Determine whether ticker is valid
            if base_currency in valid_currency and valid_volume(stock_ticker):
                filtered_tickers.append(ticker)
        except:
            print(f"{ticker} may be delisted")
        
    return filtered_tickers

filtered_stocks = filter_tickers(tickers_lst)

AGN may be delisted
CELG may be delisted
MON may be delisted
RTN may be delisted


In [7]:
# Download stock data for filtered stocks
stock_data = yf.download(tickers=filtered_stocks, interval="1d", start=filter_start_date, end=filter_end_date).Close
stock_data                   

[*********************100%%**********************]  36 of 36 completed


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,125.070000,162.380005,109.580002,270.260010,62.930000,85.820000,147.119995,195.389999,33.509998,272.630005,...,107.199997,128.029999,48.790001,26.320000,87.669998,163.210007,518.640015,207.580002,175.279999,44.639999
2023-01-04,126.360001,163.690002,111.209999,269.339996,63.860001,85.139999,150.539993,203.639999,34.139999,270.809998,...,111.529999,129.169998,50.610001,26.639999,88.809998,169.169998,504.500000,209.240005,177.110001,46.029999
2023-01-05,125.019997,163.490005,110.800003,262.980011,63.509998,83.120003,146.429993,204.990005,34.070000,271.589996,...,109.400002,128.789993,48.830002,26.610001,86.410004,166.929993,489.959991,203.080002,173.839996,45.669998
2023-01-06,129.619995,166.550003,112.330002,269.209991,64.550003,86.080002,150.169998,213.000000,34.410000,279.250000,...,115.339996,130.429993,49.560001,27.020000,86.370003,175.160004,490.000000,212.009995,178.949997,46.310001
2023-01-09,130.149994,161.660004,112.150002,273.750000,63.869999,87.360001,150.399994,208.570007,33.889999,274.720001,...,114.610001,131.399994,49.810001,26.959999,86.099998,176.679993,490.059998,211.460007,181.690002,46.610001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,173.440002,146.309998,94.809998,296.089996,59.869999,128.559998,144.419998,182.360001,25.469999,252.110001,...,109.389999,110.379997,71.870003,22.360001,76.830002,146.919998,525.000000,205.440002,149.320007,31.389999
2023-10-25,171.100006,145.259995,93.570000,292.679993,60.959999,121.389999,143.520004,177.729996,25.549999,246.720001,...,104.779999,109.110001,66.900002,22.280001,76.919998,141.789993,530.210022,205.220001,146.929993,31.290001
2023-10-26,166.889999,145.199997,93.980003,292.040009,60.849998,119.570000,143.339996,179.089996,26.120001,241.080002,...,105.620003,110.209999,64.529999,22.320000,77.410004,144.009995,528.359985,202.240005,138.210007,31.770000


## 3. Stock Analysis
In this step we will choose our stocks based on various measures.
- Get standard deviation of each stock
- Get the beta of each stock
- Get the expected returns of each stock
- Create a correlation matrix of all the stocks

In [8]:
std_dict = {}
betas = {}
exp_returns = {}

final_closings = pd.DataFrame()
Stocks_Final = pd.DataFrame()

## 4. Portfolio Optimization
- Create random weights for each stock, and create n number of random portfolios
- Choose the portfolio with the lowest expected returns

In [9]:
Portfolio_Final = pd.DataFrame()

def random_weights(n, min, max):
    """
    Generates a list of n number of random weights, each between min and max, that sum to 1
    """

    # TODO: Must modify this function to ensure final weightings are within min and max (currently after normalizing weights, they are not)
    # Create an array of n number of random weights that sum to 1
    weights = np.random.uniform(min, max, n)
    weights /= np.sum(weights)
    return weights

def random_portfolios(num_portfolios, closing_prices):
    """
    Generates a list of num_portfolios number of random portfolios (each stored in a dataframe) by randomly assigning weights to each stock

    Parameters:
    num_portfolios (int): Number of random portfolios to generate
    closing_prices (pd.DataFrame): Dataframe containing closing prices for each stock

    Returns:
    portfolios (dictionary): Dictionary containing the randomly generated portfolios.
                       Each portfolio is a dataframe containing the stocks' daily values in the portfolio based on their weights.
    expected_returns (dictionary): Dictionary containing the expected returns for each portfolio
    """

    # Remove NaN values from closing prices or we may experience some issues
    closing_prices.dropna(inplace=True)

    portfolios = {}
    expected_returns = {}
    weightings = {}

    # Create the random portfolios, each containing the stocks' daily values based on their weights
    for i in range(num_portfolios):
        weights = random_weights(closing_prices.shape[1], min_weight, max_weight)
        weightings[i] = weights

        investment_per_stock = weights * capital
        # Calculate how many shares to buy (based on the closing price of the first day)
        num_shares = investment_per_stock / closing_prices.iloc[0]

        # Calculate the daily value of each stock in the portfolio
        portfolio = closing_prices * num_shares
        portfolios[i] = portfolio

        # Calculate the expected return of the portfolio
        # Each row in this dataframe is the total value of the portfolio on that day
        total_portfolio_value = portfolio.sum(axis=1)
        # Calculate the returns of the portfolio
        returns = total_portfolio_value.pct_change()

        # TODO: double check expected return calculation
        expected_return = returns.mean()
        expected_returns[i] = expected_return
    
    return portfolios, expected_returns, weightings

In [10]:
num_portfolios = 1000 # Number of random portfolios to generate

# TODO: remove the variable below after testing
final_closings = stock_data.iloc[:, :10] # Get first 10 stocks just for testing

rand_portfolios, expected_returns, weightings = random_portfolios(num_portfolios, final_closings)

# Pick the portfolio with the expected return closest to zero
# Here, we just use the min function to find the smallest absolute value in the dictionary
optimal_portfolio = min(expected_returns, key=lambda x: abs(expected_returns[x]))
print(f"The optimal portfolio has an expected return of about {float(expected_returns[optimal_portfolio]):.15%}.")

optimal_weights_df = pd.DataFrame(weightings[optimal_portfolio], index=final_closings.columns, columns=["Weighting"])
print(f"\nHere are the best weights for each stock:")
display(optimal_weights_df)

print("The optimal portfolio is:")
display(rand_portfolios[optimal_portfolio])

The optimal portfolio has an expected return of about -0.000049610845920%.

Here are the best weights for each stock:


,Weighting
AAPL,0.092159
ABBV,0.036378
ABT,0.068770
ACN,0.059313
AIG,0.102571
AMZN,0.060111
AXP,0.190390
BA,0.122738
BAC,0.147524
BIIB,0.120046


The optimal portfolio is:


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB
Date,,,,,,,,,,
2023-01-03,69118.937555,27283.697809,51577.516054,44485.049414,76928.227995,45083.136099,142792.735087,92053.260040,110643.304696,90034.135251
2023-01-04,69831.846270,27503.808515,52344.729123,44333.614347,78065.098727,44725.916961,146112.140394,95940.047479,112723.442077,89433.090675
2023-01-05,69091.303768,27470.204277,52151.750691,43286.754832,77637.241493,43664.768232,142123.027780,96576.070117,112492.317101,89690.679768
2023-01-06,71633.456227,27984.356551,52871.896112,44312.215422,78908.586171,45219.720948,145753.027993,100349.784788,113614.929526,92220.341923
2023-01-09,71926.355824,27162.720502,52787.172910,45059.505058,78077.321087,45892.132500,145976.258825,98262.701166,111897.992135,90724.341793
...,...,...,...,...,...,...,...,...,...,...
2023-10-24,95850.233690,24583.554870,44625.516421,48736.689270,73187.556103,67535.398362,140172.153509,85914.492090,84096.837833,83257.548643
2023-10-25,94557.053381,24407.129417,44041.869691,48175.399490,74520.017248,63768.840398,139298.631253,83733.177567,84360.981563,81477.539380
2023-10-26,92230.426757,24397.048402,44234.851715,48070.057506,74385.547964,62812.754471,139123.917916,84373.908875,86243.012721,79614.969381


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.